In [3]:
from typing import List

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.decomposition import PCA

from sklearn.compose import ColumnTransformer

In [4]:
# Load data
url = "https://raw.githubusercontent.com/Tobias-Neubert94/adam_monk_II/master/adam_monk_II/data/Price_Data_Updated.gzip"
df = pd.read_parquet(url)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
# Rename columns to snake case and cleaner names
new_names = [
    "date",
    "temperature_berlin", "temperature_cologne", "temperature_frankfurt", "temperature_hamburg", "temperature_munich",
    "prep_berlin", "prep_cologne", "prep_frankfurt", "prep_hamburg", "prep_munich",
    "snow_berlin", "snow_cologne", "snow_frankfurt", "snow_hamburg", "snow_munich",
    "windspeed_berlin", "windspeed_cologne", "windspeed_frankfurt", "windspeed_hamburg", "windspeed_munich",
    "irradiation_berlin", "irradiation_cologne", "irradiation_frankfurt", "irradiation_hamburg", "irradiation_munich",
    "future_price",
    "gen_biomass",
    "gen_ff_browncoallignite",
    "gen_ff_coalderivedgas",
    "gen_fossilgas",
    "gen_fossilhardcoal",
    "gen_fossiloil",
    "gen_geothermal",
    "gen_hydropumpedstorage",
    "gen_hydrorunofriver",
    "gen_hydrowaterreservoir",
    "gen_nuclear",
    "gen_other",
    "gen_otherrenewable",
    "gen_solar",
    "gen_waste",
    "gen_windoffshore",
    "gen_windonshore",
]
df.columns = new_names

In [ ]:
# Set target at end of dataframe
df = df[[c for c in df.columns if c != "future_price"] + ["future_price"]]

In [ ]:
# Filter dataset for years 2015+ as generation data is not available as far back as 2003
df = df[df.date.dt.year >= 2015]

# Preprocessing

## Addressing null values

In [ ]:
print("These are the columns with null values:")
df.loc[:, df.isnull().sum() > 0].isnull().sum().sort_values(ascending=False)

We notice there are null values reported for Berlin and Munich across multiple weather measures (snow & irradiation for Berlin), (windspeed & snow for Munich).
Let's check how many dates report NULL for both technologies at the same time to consider whether some rows should be dropped. 

In [ ]:
df[
    (df.snow_berlin.isnull())
    & (df.irradiation_berlin.isnull())
][
    ["date"] + [col for col in df.columns if "berlin" in col]
]
# only six rows for berlin - fine to impute values

In [ ]:
df[
    (df.windspeed_munich.isnull())
    & (df.snow_munich.isnull())
][
    ["date"] + [col for col in df.columns if "munich" in col]
]
# only one row for munich - even better

### Snow

Values are most frequently missing for snowfall. Let's assume snowfall is relevant mostly for winter months (10-2), and research when snowfall values are missing. Ideally they should be in the spring/summer months (3-9), with an even spread across years.

In [ ]:
df[
    (df.snow_munich.isnull())
    | (df.snow_berlin.isnull())
    | (df.snow_cologne.isnull())
][["date", "snow_munich", "snow_berlin", "snow_cologne"]].groupby([df.date.dt.year, df.date.dt.month]).count()

# This looks good - across years, evenly missing mostly between months 5-9.

About 80% of the null values for snowfall are reported in summer months.
Let's take a simple approach for snow and assume NULL values can be filled with 0.

In [ ]:
for col in df.columns:
    if "snow" in col:
        df[col] = df[col].fillna(0)

### Solar irradiation in Berlin & wind speed in Munich

All of the values for irradiation in Berlin are missing in 2022.
To backfill, we could consider replacing with the values from the previous year, but we might not get an exact date because of the fact that we don't have pricing data for the weekend.

In [ ]:
df[(df.irradiation_berlin.isnull())][["date", "irradiation_berlin"]].groupby([df.date.dt.year, df.date.dt.month]).count()

In [ ]:
def shifted(df: pd.DataFrame, date_col: str, fill_col: str, period:int,) -> pd.DataFrame:
    df["shifted"] = df.groupby([df[date_col].dt.month, df[date_col].dt.day])[fill_col].shift(period)
    df[fill_col] = np.where(
        df[fill_col].isnull(),
        df["shifted"],
        df[fill_col]
    )
    df.drop(columns="shifted", inplace=True)
    return df

In [ ]:
df = shifted(df, "date", "irradiation_berlin", 1)
df = shifted(df, "date", "windspeed_munich", -1)

## Correlation

Here, I start by splitting our data into features and target to start working more on the features dataset. Note I take out the date column but add it back in later.

In [ ]:
# Determine features and target
X = df.drop(columns=["date", "future_price"])
features = list(X.columns)
y = df["future_price"]

In [ ]:
# First let's look at the correlation of all the features
sns.heatmap(X.corr(), cmap='coolwarm')

Takeaways: there are two distinct splits of data between weather metrics and power generation.

1. Weather features for given cities are highly correlated with eachother, such as temperature and irradiaton, but precipitation and snowfall are less so. Temperature and irraditiation are also correlated with eachother for a given city, which makes sense (if it's sunny, the temperature is higher, and vice-versa).
2. There are some high correlations among generation technologies too.
2. Let's split our features into two: weather and generation, remove some colinearity in those two cuts, and then consider the dataframe as a whole.

### Weather features

In [ ]:
sns.heatmap(pd.DataFrame(X).iloc[:, 0: 25].corr(), cmap='coolwarm')

Conclusion: let's make a single feature for temperature, irradiation, and windspeed, and look at the matrix again. It will be the mean across the cities we have selected.

In [ ]:
def average_weather(df: pd.DataFrame, measures: List[str]) -> pd.DataFrame:
    for measure in measures:
        col = f"{measure}_germany"
        df[col] = df[
            [c for c in df.columns if measure in c]
        ].mean(axis=1)
        df.drop(columns=[
            c for c in df.columns if measure in c and "germany" not in c
        ], inplace=True)
        df.insert(0, col, df.pop(col))
    return df

In [ ]:
X = average_weather(X, ["temperature", "irradiation", "windspeed"])

In [ ]:
sns.heatmap(X.iloc[:, :13].corr(), cmap='coolwarm')

### Generation technologies correlation

In [ ]:
sns.heatmap(X.iloc[:, 13:].corr(), cmap='coolwarm')

Initially I thought about grouping some of these technologies (hydro, coal, gas, wind, etc.), but given the correlations between technologies aren't so obvious, I opted to leave them as is and let the PCA analysis make the decision.

## Scaling & PCA analysis

In order to run a PCA analysis, the data needs to be centered around the mean, so we use the StandardScaler() to scale the data.

In [ ]:
X_features = list(X.columns)
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns=X_features)

In [ ]:
pca = PCA()
pca.fit(X)

# Access our 30 PCs 
W = pca.components_

# Print PCs as COLUMNS
W = pd.DataFrame(W.T,index=X_features, columns=[f'PC{i}' for i in range(1, 31)])
W

In [ ]:
sns.heatmap(W.corr(), cmap='coolwarm')

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.plot(pca.explained_variance_ratio_)
plt.xlabel('Principal Component'); plt.ylabel('% explained variance');

Almost 50% of the variance in the dataset can be explained in the first three axes.

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.ylim(ymin=0)
plt.title('cumulated share of explained variance')
plt.xlabel('# of principal component used');

In [ ]:
pca = PCA(n_components=25).fit(X) # Fit a PCA with  15 components
X_25 = pd.DataFrame(pca.fit_transform(X), columns=["PC" + str(i) for i in range(25)
]) # Project data into 25 dimensions
X_25

In [ ]:
# Test PCA data
X_train, X_test, y_train, y_test = train_test_split(X_25, y, test_size=0.3)
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test,y_test)

In [ ]:
# Compared to no PCA analysis with all features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

Conclusion: I think let's ignore the option to reduce features with PCA analysis, and just use it to remove correlation between features.

# Build pipeline

To build pipeline:

1. fill null values for snow columns with 0 using SimpleImputer.
2. apply shifted() method to two other columns with null values using FunctionTransformer.
3. Average temperature, windspeed and irradiation with average_weather() method, using FunctionTransformer.
4. Scale all features for PCA analysis with StandardScaler.
5. Build in PCA analysis.
6. Add back in date column?

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import PCA
import pandas as pd 
pd.set_option('display.max_columns', None)

In [ ]:
df.isna().sum()

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
X = df.drop(columns='future_price')
y = df['future_price']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('standard_scaler', StandardScaler())
])

pipeline.fit(X_train[['date']])
pipeline.transform(X_train[['date']])

In [ ]:
pipeline

In [ ]:
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),
    ('standard_scaler', StandardScaler())
])

In [ ]:
from sklearn import set_config; set_config(display='diagram')
preprocessor

In [ ]:
#apply shifted() method to two other columns with null values using FunctionTransformer.
import numpy as np
from sklearn.preprocessing import FunctionTransformer 
def shifted_method(prep_berlin, shift_value=0):
    shifted_prep_berlin=shift_prep_berlin, shift_value=0
    return shifted_prep_berlin 
shift_value=0

#transformer=FunctionTransformer (func=shifted_method, kw=args={'shift_value'})    
    
#prep_berlin_imputer = SimpleImputer(strategy="constant", fill_value=0)
#prep_cologne_imputer = SimpleImputer(strategy="constant", fill_value=0)

In [ ]:
#Average temperature, windspeed and irradiation with average_weather() method, using FunctionTransformer.

#  X = average_weather(X, ["temperature", "irradiation", "windspeed"])
#  with function transformer 

#calculate the mean (=average of all the data) of each weather parameter 

def mean1(temperature_berlin):
    return(float(sum(temperature_berlin)) / len(temperature_berlin))
def mean(temperature_cologne):
    return(float(sum(temperature_cologne)) / len(temperature_cologne))
def mean3(temperature_frankfurt):
    return(float(sum(temperature_frankfurt)) / len(temperature_frankfurt))
def mean4(temperature_hamburg):
    return(float(sum(temperature_hamburg)) / len(temperature_hamburg))
def mean5(temperature_munich):
    return(float(sum(temperature_munich)) / len(temperature_munich))
def mean(temperature_temperature):
    return(float(sum(mean1+mean2+mean3+mean4+mean5)) / len(mean1+mean2+mean3+mean4+mean5))

In [ ]:
#another method 

import pandas as pd
df.groupby(['temperature_berlin']).mean()

import pandas as pd
df.groupby(['temperature_cologne']).mean()

import pandas as pd
df.groupby(['temperature_frankfurt']).mean()

import pandas as pd
df.groupby(['temperature_hamburg']).mean()

import pandas as pd
df.groupby(['temperature_munich']).mean()

# create a new column with the average of the averages Temperature of the 5 cities 

df = pd.DataFrame({'average_temperature':[temperature_berlin, temperature_cologne, temperature_frankfurt, 
                                          temperature_hamburg, temperature_munich]})
df['average_temperature'] = df['temperature_berlin'] + df['Cost'])

In [ ]:
df.set_index('date')

In [ ]:
#Scale all features for PCA analysis with StandardScaler
scaler = StandardScaler()
df=df.set_index('date')
scaler.fit(df)
scaled_df = scaler.transform(df)
pca = PCA()
principal_components = pca.fit_transform(scaled_df)

In [ ]:
#Build in PCA analysis
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(df)
transformed_data = pca.transform(df)

In [ ]:
'''#fit pipeline to the training set 
pipeline.fit(X_train)
#tranform both the training set and the test set 
X_train_transformed = pd.DataFrame(pipeline.transform(X_train))
X_train_transformed
X_test_transformed = pd.DataFrame(pipeline.transform(X_test))
X_test_transformed
#standardize X-test and x_train 
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
'''

In [ ]:
snow_imputer = SimpleImputer(strategy="constant", fill_value=0)

In [ ]:
preprocessor = ColumnTransformer([
    ("snow_transformer", snow_imputer, [col for col in df.columns if "snow" in col]),
],
    remainder="passthrough",)
preprocessor

In [ ]:
preprocessor.get_feature_names_out()

In [ ]:
preprocessor = ColumnTransformer([
    ("prep_berlin_transformer", prep_berlin_imputer, [col for col in df.columns if "prep_berlin" in col]),
], remainder="passthrough",)
    ("prep_cologne_transformer", prep_cologne_imputer, [col for col in df.columns if "prep_cologne" in col]),
], remainder="passthrough",)
preprocessor

# Data visualisation 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_new = df[['date', 'future_price']]
print(df_new)

In [ ]:
plt.plot(df['date'], df['future_price'])
plt.xlabel('date')
plt.ylabel('future_price')
plt.title('Evolution of prices')
plt.show()

In [ ]:
# Simple graph of evolution of future prices 
plt.plot(date_x, future_price_y)
plt.ylabel('Future electricity prices')
plt.title('Evolution of future electricity pricesin Germany')
plt.show()

# Appendix: Charting

In [ ]:
def plot_weather_patterns(city: str):

    temp = [col for col in df.columns if city in col and "temperature" in col]
    prep = [col for col in df.columns if city in col and "prep" in col]
    wind = [col for col in df.columns if city in col and "windspeed" in col]
    snow = [col for col in df.columns if city in col and "snow" in col]
    irr = [col for col in df.columns if city in col and "irradiation" in col]

    # Start figure
    plt.figure(figsize=(10,15))

    # Temperature
    plt.subplot(5, 1, 1)
    plt.plot(df.date, df[temp], c="black", linewidth=0.5)
    plt.title("Average temperature")
    # Precipitation
    plt.subplot(5, 1, 2)
    plt.plot(df.date, df[prep], c='black', linewidth=0.5)
    plt.title("Precipitation")
    # Wind speed
    plt.subplot(5, 1, 3)
    plt.plot(df.date, df[wind], c='black', linewidth=0.5)
    plt.title("Wind speed")
    # Snow fall
    plt.subplot(5, 1, 4)
    plt.plot(df.date, df[snow], c='black', linewidth=0.5)
    plt.title("Snow")
    # Solar irradiation
    plt.subplot(5, 1, 5)
    plt.plot(df.date, df[irr], c='black', linewidth=0.5)
    plt.title("Solar irradiation")
    
    plt.show();

In [ ]:
plot_weather_patterns("munich")